In [5]:
# clean_data
import os
import re

folder = 'dataHAX/data/'
for filename in os.listdir(folder):
    f = os.path.join(folder, filename)
    row = 0
    with open(f, 'r', encoding='utf-8') as file:
        for line in file:
            if line == "\n":
                continue
            else:
                print(line, end="")
        break

dataHAX/data/out__2022-09-10_23-59-59.log
10.09.2022 00:00:01.0 : 📢 : 🔵 Гол для синих! | 3:2 – 01:51 | ⚽Dorfixz
10.09.2022 00:00:15.2 : 📢 : 🔴 Гол для красных! | 4:2 – 02:01 | ⚽FM | Tekashe 
10.09.2022 00:00:17.5 : Red team won the match
10.09.2022 00:00:17.7 : 📢 : 🔴🏆 Лимит по очкам! Победа красных со счётом 4:2! Лучшие по Г+П: FM | Tekashe  (2+1), Dorfixz (2+0)
10.09.2022 00:00:22.5 : Game stopped 
10.09.2022 00:00:22.7 : 📢 : Игрок матча – Dorfixz! Согласны?😅
10.09.2022 00:00:34.7 : Stadium "Futsal 3x3-4x4" loaded 
10.09.2022 00:00:34.7 : Dorfixz was moved to Spectators 
10.09.2022 00:00:34.7 : Enner was moved to Spectators 
10.09.2022 00:00:34.7 : MESONDAR was moved to Spectators 
10.09.2022 00:00:34.7 : FM | Tekashe  was moved to Spectators 
10.09.2022 00:00:34.7 : extra 0 enjoyer was moved to Blue 
10.09.2022 00:00:34.7 : GównoZDupska was moved to Blue 
10.09.2022 00:00:34.7 : Dorfixz was moved to Blue 
10.09.2022 00:00:34.7 : FM | Tekashe  was moved to Blue 
10.09.2022 00:00:34.7 :

In [ ]:
# p = re.compile("(\d+\.?){3} \d+:\d+:\d+.\d : Game stopped")
# bool(p.match("10.09.2022 15:38:22.0 : Game started"))

In [115]:
def match_finder(file):
    # "dataHAX/data/out__2022-09-10_23-59-59.log"
    list_of_matches = []
    game_stopped = re.compile("(\d+\.?){3} \d+:\d+:\d+.\d : Game stopped")
    current_match = []

    for line in file:
        if game_stopped.match(line):
            if current_match[1:] != [] : 
                list_of_matches.append(current_match[1:])
            current_match = []
        elif line.strip():
                current_match.append(line.strip())    
    return list_of_matches


def starting_lineup(match):
    players = {'Red' : [], 'Blue': []}
    game_started = re.compile("(\d+\.?){3} \d+:\d+:\d+.\d : Game started")
    player_side = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : (.*) was moved to (\w{3,4})$")
    for event in match:
        if game_started.match(event):
            break
        if player_side.match(event):
            players[player_side.match(event).group(2)].append(player_side.match(event).group(1))
    return players


def match_result(match):
    winner = 0
    the_event = match[-1]    
    for m in match:
        if m.find(' : 📢 : ') != -1:
            the_event = m            
            
    score_limit = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : 📢 : (.?)🏆 Лимит по очкам! .*")
    time_limit = re.compile("(?:\d+\.?){3} \d+:\d+:\d+.\d : 📢 : (.?)🏆 Время вышло.*")
    if score_limit.match(the_event):
        if score_limit.match(the_event).group(1) == "🔵":
            winner = 2
        else: 
            winner = 1
    elif time_limit.match(the_event):
        if time_limit.match(the_event).group(1) == "🔵":
            winner = 2
        else: 
            winner = 1
    return winner

def get_goals(match, players):
    for line in match:
        for name in players:
            if '⚽'+name in line and "Автогол" not in line: 
                    stats[name][2] += 1


def update_stats(winner, players, m):
    if winner > 0 and len(players['Red']) == 4:
        for p in players['Red'] + players['Blue']:
            if p not in stats.keys():
                stats[p] = [0, 1, 0]
            else:
                stats[p][1] += 1
        if winner == 1:
            for p in players['Red']:
                stats[p][0] += 1
        else: 
            for p in players['Blue']:
                stats[p][0] += 1
        
        get_goals(m, players['Red'] + players['Blue'])


In [116]:
import os
import re
import sys


def match_analyser(match):
    m_players = starting_lineup(match)
    m_result = match_result(match)
    return m_result, m_players


global stats
stats = {}

folder = 'dataHAX/data/'
for filename in os.listdir(folder):
    f = os.path.join(folder, filename)    
    with open(f,'r', encoding='utf-8') as file:
        match_list = match_finder(file)
        for m in match_list:
            win, players = match_analyser(m)
            update_stats(win, players, m)


sorted_stats = sorted(stats.items(), key=lambda kv: kv[1][1], reverse=True)

In [96]:
match_list[213]

['17.09.2022 20:14:23.7 : gh was moved to Spectators by Окa',
 '17.09.2022 20:14:23.7 : Шахтар Донецьк was moved to Spectators by Окa',
 '17.09.2022 20:14:23.7 : йо13 💙💛 was moved to Spectators by Окa',
 '17.09.2022 20:14:23.8 : AzoT was moved to Spectators by Окa',
 '17.09.2022 20:14:26.7 : Окa was moved to Blue by Окa',
 '17.09.2022 20:14:34.3 : Шахтар Донецьк was moved to Blue by Окa',
 '17.09.2022 20:14:36.6 : YR was moved to Blue by Окa',
 '17.09.2022 20:14:37.6 : AzoT was moved to Blue by Окa',
 '17.09.2022 20:14:38.2 : Game started by Окa',
 '17.09.2022 20:19:30.7 : 📢 : 🔴 Гол для красных! | 1:0 – 04:45 | ⚽Швец. ← Neymar                   👟',
 '17.09.2022 20:19:33.0 : Red team won the match',
 '17.09.2022 20:19:33.2 : 📢 : 🔴🏆 Время вышло. Победа красных со счётом 1:0! Лучшие по Г+П: Швец. (1+0), Neymar                    (0+1)',
 '17.09.2022 20:19:37.9 : Окa was moved to Spectators by Окa']

In [122]:
for s in sorted_stats:
    print(s[0], round(s[1][0]/s[1][1],2), s[1][1], s[1][2], sep=';;')

SNEG;;0.45;;350;;63
3oomer);;0.54;;348;;158
УАрик;;0.5;;297;;125
контрнаступ;;0.48;;288;;49
lfc;;0.65;;277;;183
Журналист;;0.39;;260;;137
 ;;0.55;;258;;101
sonic;;0.59;;244;;36
xXx;;0.66;;240;;96
technoviking;;0.53;;237;;51
fade;;0.53;;218;;98
Ted Lasso;;0.57;;208;;84
extra 0 enjoyer;;0.63;;196;;117
Dorfixz;;0.53;;194;;81
AzoT;;0.39;;193;;53
корж;;0.53;;189;;48
Inzaghi;;0.54;;166;;64
Фоден;;0.5;;163;;16
sinchiroca;;0.61;;160;;22
Khvicha Kvaratskhelia;;0.35;;157;;42
 Agbonlahor 11;;0.73;;150;;155
AzoT💙💛;;0.46;;149;;58
mali;;0.54;;145;;116
cop;;0.45;;145;;21
BLACKPINK;;0.48;;144;;50
Nigger;;0.51;;143;;14
ХуТоРоК;;0.42;;140;;47
Poop;;0.51;;139;;21
Koldi_;;0.39;;132;;36
ChPoKvPuPoK;;0.67;;129;;88
Эстетика ануса;;0.67;;129;;92
zuu.;;0.46;;128;;41
Tekashe💙💛 ;;0.69;;123;;30
Broadway182;;0.46;;123;;11
Шкарпетка;;0.54;;122;;48
Шахтар Донецьк;;0.38;;122;;26
pro0ch;;0.5;;121;;133
69;;0.5;;119;;84
(^^,);;0.52;;118;;32
YR;;0.57;;116;;57
Sad_Bodya;;0.47;;116;;41
Neymar                   ;;0.46;;114;